This part will introduct the def functions we used. We will use some sprcific example for clear explaination.

In [4]:
# Import the necessary packages
import music21
from music21 import  *
import shutil
from collections import Counter, defaultdict
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import sys, re, itertools, random, os
from functions import *

## ATTENTION
***We will show how to use the functions below by using music Billy-Boy.mid in function-example folder as an example. However, if you want to try it by yourself, you should delete the original files(such as .txt,.chord and .node files) except the .mid file, otherwise the new information will be written in the original files repeatly.***

### Function 1
**TransferMidToChordTxt** is used to transfer the input file (.mid music) to .txt format with the track information.

In [43]:
def TransferMidToChordTxt(file_path):
    try:
        components = readMidiFile(file_path=file_path)
        startPiano = True 
        printRatio = True
        printHighest = True
        printColumns = True
        output = ""
        notes = ""
        chords = ""
        for component in components:
            if startPiano and type(component) is music21.chord.Chord:
                tmp = component.fullName + "," + component.pitchedCommonName + "," + str(component.quarterLength) + "," + str(
                    component.offset)
                output += tmp + "\n"
                print(tmp)
            elif type(component) is music21.meter.TimeSignature and printRatio:
                tmp = str(component.ratioString)
                output += tmp + "\n"
                print(tmp)
                printRatio = False
            elif type(component) is music21.stream.Score and printHighest:
                tmp = component.highestTime
                output += str(tmp) + "\n"
                print(tmp)
                printHighest = False
            elif not printHighest and not printRatio and printColumns:
                tmp = "FullName,CommonName,Len,Offset"
                output += tmp + "\n"
                print(tmp)
                printColumns = False
        with open((file_path.split(".mid")[0] + "_chord.txt"), 'w') as f:
            f.write(output)
    except:
        pass

In [45]:
path = "function-example/Billy-Boy.mid"
TransferMidToChordTxt(path)

### Function 2
**ExtractNodeAndChord** is used to extract the chord and node information from the input file (.txt file with track information).

In [30]:
def ExtractNodeAndChord(file_path, split_length,out_node_path,out_chord_path):
    read_data = None
    full_name = ""
    common_name = ""
    count_length = 0
    with open(file_path, 'r') as f:
        read_data = f.readlines()
    for line in read_data[1:]:
        if line == "" or "{" not in line:
            continue
        count_length += 1
        sl = line.split(",")
        i  = sl[0].index("{")
        j  = sl[0].index("}")
        full_name += sl[0][i:j+1].replace(" ", "_") + " "
        temp = sl[1].split(" ")[0] + " "
        common_name += sl[1].split(" ")[0].split("-")[0] + " "
        if split_length is not None and count_length == split_length:
            full_name += "\n"
            common_name += "\n"
            count_length = 0

    if full_name.endswith("\n"):
        full_name = full_name[:-1]
    if common_name.endswith("\n"):
        common_name = common_name[:-1]
    print ("--------This is tpye.node--------")
    print (full_name)
    print ("\n")
    print ("--------This is type.chord--------")
    print (common_name)
    with open(out_node_path, 'a') as f:
        f.write(full_name + '\n')
    with open(out_chord_path, 'a') as f:
        f.write(common_name + '\n')

In [31]:
file_path = "function-example/Billy-Boy_chord.txt"
split_length = 20
out_node_path = "function-example/Billy-Boy_chord.node"
out_chord_path = "function-example/Billy-Boy_chord.chord"
ExtractNodeAndChord(file_path, split_length,out_node_path,out_chord_path)

--------This is tpye.node--------
{E-flat_in_octave_3_|_C-sharp_in_octave_3} {F-sharp_in_octave_2_|_E-flat_in_octave_3} {F-sharp_in_octave_2_|_C-sharp_in_octave_2_|_E-flat_in_octave_3} {F-sharp_in_octave_2_|_E-flat_in_octave_3} {F-sharp_in_octave_2_|_C-sharp_in_octave_2_|_E-flat_in_octave_3} {F-sharp_in_octave_2_|_E-flat_in_octave_3} {F-sharp_in_octave_2_|_C-sharp_in_octave_2_|_E-flat_in_octave_3} {F-sharp_in_octave_2_|_E-flat_in_octave_3} {F-sharp_in_octave_2_|_C-sharp_in_octave_2_|_E-flat_in_octave_3} {F-sharp_in_octave_2_|_E-flat_in_octave_3} {F-sharp_in_octave_2_|_C-sharp_in_octave_2_|_E-flat_in_octave_3} {F-sharp_in_octave_2_|_E-flat_in_octave_3} {F-sharp_in_octave_2_|_C-sharp_in_octave_2_|_E-flat_in_octave_3} {F-sharp_in_octave_2_|_E-flat_in_octave_3} {F-sharp_in_octave_2_|_C-sharp_in_octave_2_|_E-flat_in_octave_3} {F-sharp_in_octave_2_|_E-flat_in_octave_3} {F-sharp_in_octave_2_|_C-sharp_in_octave_2_|_E-flat_in_octave_3} {F-sharp_in_octave_2_|_E-flat_in_octave_3} {F-sharp_in_octa

In [32]:
print ("Total node:",len(open("function-example/Billy-Boy_chord.chord",'r').readlines()))
print ("Total chord:",len(open("function-example/Billy-Boy_chord.node",'r').readlines()))

Total node: 32
Total chord: 32


### Function 3
**RemoveDuplicateVocab** is used to remove the deplicate word in a vocabulary. The input file is a vocabulary file(.chord/.node) and the output file remove the duplicate words. 

In [65]:
vob = set()
with open("function-example/Billy-Boy_chord.chord", 'r') as f:
    read_data = f.readlines()
for lines in read_data:
    for c in lines.split(" "):
        if c:
            vob.add(c)
print (vob)

{'F#2', '\n', 'D4', 'C4', 'G3', 'A4', 'C5', 'E4', 'C#4', 'B', 'D2', 'E', 'E3', 'F3', 'C#3', 'G4', 'A3'}


In [66]:
with open("function-example/Billy-Boy_chord.vocab", 'w') as f:
    for c in vob:
        if c:
            f.write(c+'\n')

In [67]:
def RemoveDuplicateVocab(vocabFile):
    words = []
    num = 0
    with open(vocabFile, 'r') as f:
        for line in f:
            line = line.strip()
            if line not in words:
                words.append(line)
            else:
                num += 1
                print("Remove",num ,"duplicate words!")
        f.close()
    try:
        os.remove(vocabFile)
    except OSError:
        pass
    with open(vocabFile, 'w') as f:
        for word in words:
            f.write(word +'\n')
        f.close()

In [68]:
vocabFile = "function-example/Billy-Boy_chord.vocab"
RemoveDuplicateVocab(vocabFile)

Find 1 duplicate words!


### Function 4
**RemoveEmptyLine** is used to remove the empty lines in a file.

In [14]:
def RemoveEmptyLine(file_path):
    lines = []
    num = 0
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if len(line) == 0:
                num += 1
                continue
            else:
                lines.append(line)
        f.close()
    try:
        os.remove(file_path)
    except OSError:
        pass
    with open(file_path, 'w') as f:
        for line in lines:
            f.write(line+'\n')
        f.close()
    print("Remove",num ,"empty lines!")

In [15]:
file_path = "function-example/Billy-Boy_chord.vocab"
RemoveEmptyLine(file_path)

Remove 1 empty lines!


### Function 5
**PrintNumLines** is used to print sever lines in a file.

In [25]:
def PrintNumLines(file_path,num):
    lines = []
    i = 1
    with open(file_path,'r') as f:
        for line in f:
            line = line.strip()
            if i <= num:
                print(line)
                i += 1
                continue
        f.close()

In [26]:
file_path = "function-example/Billy-Boy_chord.vocab"
num = 5
PrintNumLines(file_path,num)

F#2
D4
C4
G3
A4
